### Merge Title and Comments

In [1]:
import pandas as pd
from tqdm import tqdm


In [2]:
train_data = pd.read_csv("./data/text_metadata/multimodal_only_samples/multimodal_train.tsv", sep='\t')
test_data = pd.read_csv("./data/text_metadata/multimodal_only_samples/multimodal_test_public.tsv", sep='\t')

In [3]:
# train_data.head(5)

In [4]:
train_data_selected = train_data[["clean_title", "id", "image_url", "2_way_label", "3_way_label", "6_way_label"]]
test_data_selected = test_data[["clean_title", "id", "image_url", "2_way_label", "3_way_label", "6_way_label"]]

In [5]:
comments_data = pd.read_csv("./data/comments/all_comments.tsv", sep='\t')

/tmp/ipykernel_74111/4234283532.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  comments_data = pd.read_csv("./data/comments/all_comments.tsv", sep='\t')


In [6]:
comments_data = comments_data[["body", "submission_id"]]

In [7]:
def search_comments(key):
    comments_list = []
    result = comments_data.loc[comments_data.submission_id == key]
    if not result.empty:
        for idx, row in result.iterrows():
            comments_list.append(row.body)
    return comments_list

#### 2_way_label dataset

In [12]:
top_negative_train_data = train_data_selected[train_data_selected["2_way_label"] == 0].head(7500)
top_postive_train_data = train_data_selected[train_data_selected["2_way_label"] == 1].head(7500)
train_data_2_classes = pd.concat([top_negative_train_data, top_postive_train_data])

In [13]:
top_negative_test_data = test_data_selected[test_data_selected["2_way_label"] == 0].head(1500)
top_postive_test_data = test_data_selected[test_data_selected["2_way_label"] == 1].head(1500)
test_data_2_classes = pd.concat([top_negative_test_data, top_postive_test_data])

In [14]:
train_csv_list = []
test_csv_list = []
title = ["id", "text_comments", "text_only", "comments_only", "2_way_label"]
base_dir = "./data/processed_data/2_classes_big/"

In [15]:
progress_bar = tqdm(train_data_2_classes.iterrows(), total = len(train_data_2_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, row["2_way_label"]]
    train_csv_list.append(single_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 15000/15000 [2:00:12<00:00,  2.08it/s]


In [17]:
train_csv_file = pd.DataFrame(columns = title, data = train_csv_list)
train_csv_file.to_csv(base_dir + "train_data.csv")

In [18]:
train_csv_file

,id,text_comments,text_only,comments_only,2_way_label
0,98pbid,this concerned sink with a tiny hat[SEP]“Does ...,this concerned sink with a tiny hat,“Does this hat make my head look too big?”[SEP...,0
1,8gnet9,i found a face in my sheet music too[SEP]That ...,i found a face in my sheet music too,That crescendo part at the top also kind of lo...,0
2,6d50rl,major thermos[SEP],major thermos,,0
3,cgp0lmq,cutouts[SEP],cutouts,,0
4,98vrcq,jesus christ converting local teens to christi...,jesus christ converting local teens to christi...,,0
...,...,...,...,...,...
14995,cawoo4,the way this creamer settled in my coffee[SEP]...,the way this creamer settled in my coffee,"There's clearly a T-Rex nearby, take shelter.[...",1
14996,2c393j,"chewbacca court defence fails[SEP]""It was orig...",chewbacca court defence fails,"""It was originally used by the defence in the ...",1
14997,8ytbif,maryland ag probing russian oligarchs purchase...,maryland ag probing russian oligarchs purchase...,,1
14998,3jpezt,lost doll takes a vacation before finding her ...,lost doll takes a vacation before finding her ...,Lucky doll![SEP],1


In [19]:
progress_bar = tqdm(test_data_2_classes.iterrows(), total = len(test_data_2_classes))
for idx, row in progress_bar:
    comments_only = ""
    text_comments = ""
    text_comments += row["clean_title"]
    text_comments += "[SEP]"
    comments_list = search_comments(row["id"])
    for each_comment in comments_list:
        comments_only += str(each_comment)
        comments_only += "[SEP]"
        text_comments += str(each_comment)
        text_comments += "[SEP]"
        
    single_list = [row["id"], text_comments, row["clean_title"], comments_only, 
                   row["2_way_label"]]
    test_csv_list.append(single_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [24:02<00:00,  2.08it/s]


In [20]:
test_csv_file = pd.DataFrame(columns = title, data = test_csv_list)
test_csv_file.to_csv(base_dir + "test_data.csv")

In [21]:
test_csv_file

,id,text_comments,text_only,comments_only,2_way_label
0,cozywbv,stargazer[SEP],stargazer,,0
1,ctk61yw,yeah[SEP],yeah,,0
2,7o9rmx,believers hezbollah[SEP]Does Hezbollah actuall...,believers hezbollah,Does Hezbollah actually have kit that nice? Ke...,0
3,bdfxf1,the rise of italian fascism circa[SEP],the rise of italian fascism circa,,0
4,2s0xuj,look at this cool iphone case i got the other ...,look at this cool iphone case i got the other day,Bath bomb?[SEP]I actually saw a iPhone case ev...,0
...,...,...,...,...,...
2995,8xelms,refugee bartender raises funds and provides ai...,refugee bartender raises funds and provides ai...,,1
2996,bgdnq9,this banana came with its own snapchat filter[...,this banana came with its own snapchat filter,That's very appealing[SEP]Well **where the fuc...,1
2997,bu6bnp,public restroom w foot washing station[SEP]Def...,public restroom w foot washing station,Definitely for our Muslim friends and any othe...,1
2998,9i16q9,this red box behind a bus that is censoring th...,this red box behind a bus that is censoring th...,,1
